In [1]:
from pathlib import Path  # For defining dataset Paths
import sys
Root = "../../../../../.."
sys.path.append("../../../..")

In [2]:
import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from umap import UMAP
#from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

import plotly.express as px
import plotly.graph_objects as go
import itertools
from itertools import combinations

# Librep imports
from librep.utils.dataset import PandasDatasetsIO          # For quick load train, test and validation CSVs
from librep.datasets.har.loaders import (
    KuHar_BalancedView20HzMotionSenseEquivalent
)
from librep.datasets.multimodal import PandasMultiModalDataset, TransformMultiModalDataset, WindowedTransform
from librep.transforms.fft import FFT
from librep.transforms. stats import StatsTransform
from librep.utils.workflow import SimpleTrainEvalWorkflow, MultiRunWorkflow
from librep.estimators import RandomForestClassifier, SVC, KNeighborsClassifier
from librep.metrics.report import ClassificationReport
from librep.transforms.resampler import SimpleResampler

2022-12-19 16:48:26.194363: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-19 16:48:26.194379: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Datasets to train the manifold

In [4]:
labels_activity = {
    0: "sit",
    1: "stand",
    2: "walk",
    3: "stair up",
    4: "stair down",
    5: "run",
    6: "stair up and down",
}

classes = list(labels_activity.keys())
print(labels_activity)

labels_dataset = {
    'KuHar': 'KuHar', 
    'RealWorld': 'RealWorld',
    'MotionSense': 'MotionSense',
    'ExtraSensory': 'ExtraSensory',
    'WISDM': 'WISDM',
    'UCI': 'UCI',
}

{0: 'sit', 1: 'stand', 2: 'walk', 3: 'stair up', 4: 'stair down', 5: 'run', 6: 'stair up and down'}


## Load Data

In [5]:
# Load all datasets, creating PandasMultiModalDatasets with the correct pre-defined windows
loader = KuHar_BalancedView20HzMotionSenseEquivalent(
    Root+"/data/views/AllDatasets/balanced_20Hz_filtered", 
#     Root+"/data/views/KuHar/balanced_20Hz_motionsense_equivalent",
    download=False)
train_data, test_data = loader.load(concat_train_validation=True, label="standard activity code")
train_data, test_data

(PandasMultiModalDataset: samples=54522, features=360, no. window=6, label_columns='standard activity code',
 PandasMultiModalDataset: samples=6880, features=360, no. window=6, label_columns='standard activity code')

In [6]:
# dimensions_umap = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 50, 100, 150, 200, 250, 300, 360]
dimensions_umap = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 50, 100, 150, 180]

datasets = ['KuHar', 'RealWorld', 'MotionSense', 'WISDM', 'UCI']

In [7]:
columns = ['Classifier', 'Umap dimension', 'Dataset']

metrics = ['accuracy', 'f1 score (weighted)']
stats = ['mean', 'std']
columns += [metric + ' - ' + stat
            for metric in metrics
            for stat in stats]

metrics_class = ['f1-score', 'precision', 'recall', 'support']
columns += [
    metric + ' - ' + stat + ' - ' + activity
    for metric in metrics_class
    for stat in stats
    for activity in labels_activity.values()
]

columns, len(columns)
df_results = {column: [] for column in columns}

results_dict = {
    'RandomForest': {}, 
    'SVC': {}, 
    'KNN': {}
}
for classifier in results_dict.keys():
    results_dict[classifier] = {
        'Umap dimension': [],
        'Dataset': [],
        'result': []
    }

In [8]:
def create_data_multimodal(data):
    # Features to select
    features = [
        "accel-x",
        "accel-y",
        "accel-z",
        "gyro-x",
        "gyro-y",
        "gyro-z"
    ]

    # Creating the datasets

    # Data
    data_multimodal = PandasMultiModalDataset(
        data,
        feature_prefixes=features,
        label_columns="standard activity code",
        as_array=True
    )

    return data_multimodal

In [9]:
def evaluate(dimension, dataset, train, test, evaluators, df, results_dict, labels_activity, metrics_class, 
             reporter):
# The reporter will be the same

    fft_transform = FFT(centered=True)

    if dimension == 180:
        transformer = TransformMultiModalDataset(transforms=[fft_transform], 
                                                 new_window_name_prefix="fft.")

    else:
        transformer_fft = TransformMultiModalDataset(transforms=[fft_transform], 
                                                 new_window_name_prefix="reduced.")

        umap = UMAP(n_components=dimension, random_state=42)
        train_fft = transformer_fft(train)

        umap.fit(train_fft[:][0])

        umap_transform = WindowedTransform(
            transform=umap, fit_on=None, transform_on="all"
        )

        transformer = TransformMultiModalDataset(transforms=[fft_transform, umap_transform], 
                                                 new_window_name_prefix="reduced.")
    train_fft = transformer(train)
    test_fft = transformer(test)

    for estimator, evaluator in evaluators.items():
        multi_run_experiment = MultiRunWorkflow(
            workflow=evaluator['experiment'], 
            num_runs=evaluator['num_runs'],
            debug=False)

        results = multi_run_experiment(train_fft, test_fft)
        results_dict[estimator]['Umap dimension'].append(dimension)
        results_dict[estimator]['Dataset'].append(dataset)
        results_dict[estimator]['result'].append(results)

        df['Classifier'].append(estimator)
        df['Umap dimension'].append(dimension)
        df['Dataset'].append(dataset)

        df['accuracy - mean'].append(
            np.average(
                [res["result"][0]["accuracy"] for res in results["runs"]]
            )
        )
        df['accuracy - std'].append(
            np.std(
                [res["result"][0]["accuracy"] for res in results["runs"]]
            )
        )
        df['f1 score (weighted) - mean'].append(
            np.average(
                [res["result"][0]["f1 score (weighted)"] for res in results["runs"]]
            )
        )
        df['f1 score (weighted) - std'].append(
            np.std(
                [res["result"][0]["f1 score (weighted)"] for res in results["runs"]]
            )
        )

        labels = test.data['standard activity code'].unique()
        for metric in metrics_class:
            for index, activity in labels_activity.items():
                df[f'{metric} - mean - {activity}'].append(
                    np.average(
                        [res['result'][0]['classification report'][str(index)][metric] for res in results["runs"]]
                    )
                ) if index in labels else  df[f'{metric} - mean - {activity}'].append(np.nan)

                df[f'{metric} - std - {activity}'].append(
                    np.std(
                        [res['result'][0]['classification report'][str(index)][metric] for res in results["runs"]]
                    )
                ) if index in labels else  df[f'{metric} - std - {activity}'].append(np.nan)
    return df, results_dict

In [10]:
start = time.time()
reporter = ClassificationReport(
    use_accuracy=True,
    use_f1_score=True,
    use_classification_report=True,
    use_confusion_matrix=True,
    plot_confusion_matrix=False,
#     normalize='true',
#     display_labels=labels,
)

evaluators = {
    'RandomForest':
    {
        'experiment':
        SimpleTrainEvalWorkflow(
            estimator=RandomForestClassifier, 
            estimator_creation_kwags ={'n_estimators':100}, 
            do_not_instantiate=False, 
            do_fit=True, 
            evaluator=reporter),
        'num_runs':
        10

    },
    'SVC':
    {
        'experiment':
        SimpleTrainEvalWorkflow(
            estimator=SVC, 
            estimator_creation_kwags ={'C':3.0, 'kernel':"rbf"} , 
            do_not_instantiate=False, 
            do_fit=True, 
            evaluator=reporter),
        'num_runs':
        1
    },
    'KNN':
    {
        'experiment':
        SimpleTrainEvalWorkflow(
            estimator=KNeighborsClassifier, 
            estimator_creation_kwags={'n_neighbors' :1}, 
            do_not_instantiate=False, 
            do_fit=True, 
            evaluator=reporter),
        'num_runs':
        1
    }
}

train_data.data['standard activity code'] = train_data.data['standard activity code'].astype('int')
test_data.data['standard activity code'] = test_data.data['standard activity code'].astype('int')
k=1
for dataset in datasets:

    train = train_data.data[train_data.data['DataSet'].isin([dataset])]
    train = create_data_multimodal(train)

    test = test_data.data[test_data.data['DataSet'].isin([dataset])]
    test = create_data_multimodal(test)

    new_start = time.time()
    for dimension in dimensions_umap:
        df_results, results_dict = evaluate(dimension, dataset, train, test, evaluators, df_results, 
                                            results_dict, labels_activity, metrics_class, reporter)
        new_end = time.time()
        print(f'Combination: {k} \t Time of execution: {int(new_end - new_start) // 60} minutes and {int(new_end - new_start) % 60} seconds')
        k+=1

end = time.time()
total = int(end - start)
print(f'Time of execution: {total} seconds')
print(f'Time of execution: {total // 60} minutes and {total % 60} seconds')
print(f'Time of execution: {(total // 86400)} days, {(total // 3600) % 24} hours, {(total // 60) % 60} minutes and {total % 60} seconds')

Combination: 1 	 Time of execution: 0 minutes and 19 seconds
Combination: 2 	 Time of execution: 0 minutes and 31 seconds
Combination: 3 	 Time of execution: 0 minutes and 43 seconds
Combination: 4 	 Time of execution: 0 minutes and 56 seconds
Combination: 5 	 Time of execution: 1 minutes and 10 seconds
Combination: 6 	 Time of execution: 1 minutes and 23 seconds
Combination: 7 	 Time of execution: 1 minutes and 37 seconds
Combination: 8 	 Time of execution: 1 minutes and 51 seconds
Combination: 9 	 Time of execution: 2 minutes and 6 seconds
Combination: 10 	 Time of execution: 2 minutes and 22 seconds
Combination: 11 	 Time of execution: 2 minutes and 39 seconds
Combination: 12 	 Time of execution: 2 minutes and 59 seconds
Combination: 13 	 Time of execution: 3 minutes and 22 seconds
Combination: 14 	 Time of execution: 3 minutes and 54 seconds
Combination: 15 	 Time of execution: 4 minutes and 39 seconds
Combination: 16 	 Time of execution: 5 minutes and 34 seconds
Combination: 17 	 

In [21]:
train.data.iloc[:,1:-3]

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-50,gyro-z-51,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59
13188,0.156566,-1.424124,-2.905373,-1.449460,-1.068465,-1.123434,-1.195326,-0.833242,1.702248,2.030561,...,0.080595,-0.112580,0.156026,0.313386,0.000744,-0.047502,-0.569485,-0.652966,0.050269,0.477185
13189,-0.537229,-0.186480,1.051567,0.605424,-0.185871,3.032825,4.907514,4.161293,1.228430,-0.243705,...,0.307554,0.768617,0.822427,0.841789,0.214277,-0.155207,-0.054555,0.275682,0.461437,-0.171866
13190,-2.151572,-2.340571,0.602388,2.951933,1.112684,3.037523,-0.053386,-1.381455,-0.637672,-2.754747,...,0.239563,0.095893,0.056733,-0.278544,-0.050879,0.363474,0.356592,0.452005,0.304909,-0.237935
13191,-0.121271,-1.496275,-1.492793,-1.547828,-0.905671,-0.681165,-0.371455,0.291828,1.207175,1.622875,...,-0.232457,-0.074255,0.416932,0.290052,0.337902,0.202508,-0.316462,-0.537158,-0.476181,-0.320307
13192,1.557988,1.860748,1.597169,2.202641,3.586844,1.869756,-1.940422,-3.407308,-3.873758,-2.844197,...,-0.143105,0.161871,0.269892,0.205870,-0.161104,-0.067245,-0.006595,-0.044617,-0.235705,0.503788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53035,-0.026208,-0.024057,-0.196275,-0.139232,-0.018196,-0.031718,-0.064043,-0.024471,-0.033220,0.005276,...,-0.008467,-0.024507,-0.035031,-0.013156,0.002951,0.015558,0.016599,0.009087,0.002833,0.004040
53036,-0.125996,0.076052,-0.152721,-0.113567,-0.076924,0.261904,0.006202,0.149830,-0.070275,-0.137767,...,-0.117224,-0.145218,-0.158068,-0.169206,-0.138596,-0.130194,-0.112169,-0.091970,-0.074266,-0.100618
53037,-0.015568,-0.002566,-0.021435,-0.005103,-0.019504,0.006622,-0.036400,-0.016476,0.013177,0.025060,...,-0.000391,0.002910,0.002480,-0.002914,-0.003599,-0.005550,-0.008568,-0.012022,-0.003929,0.008076
53038,0.002989,-0.074446,-0.064812,-0.093166,-0.091162,-0.102577,-0.128314,-0.114664,-0.079710,-0.081442,...,-0.011717,-0.008635,0.000560,-0.016810,-0.021231,-0.084481,0.003491,0.123360,-0.004188,-0.140925


In [11]:
df_results = pd.DataFrame(df_results)
df_results.loc[df_results['Classifier'] == 'RandomForest']
# df_results.dropna(axis=1)

,Classifier,Umap dimension,Dataset,accuracy - mean,accuracy - std,f1 score (weighted) - mean,f1 score (weighted) - std,f1-score - mean - sit,f1-score - mean - stand,f1-score - mean - walk,...,support - mean - stair down,support - mean - run,support - mean - stair up and down,support - std - sit,support - std - stand,support - std - walk,support - std - stair up,support - std - stair down,support - std - run,support - std - stair up and down
0,RandomForest,1,KuHar,0.654472,0.000000,0.652102,1.110223e-16,0.683544,0.705882,0.652174,...,39.0,34.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
3,RandomForest,2,KuHar,0.743496,0.009697,0.742913,9.575583e-03,0.786043,0.774419,0.748931,...,38.3,36.0,NaN,1.720465,1.720465,0.600000,0.830662,1.100000,0.000000,NaN
6,RandomForest,3,KuHar,0.765854,0.006350,0.764473,6.159894e-03,0.852841,0.819236,0.722910,...,37.8,34.8,NaN,1.345362,1.345362,0.458258,0.830662,0.748331,0.400000,NaN
9,RandomForest,4,KuHar,0.760569,0.004244,0.759171,4.217539e-03,0.823757,0.790392,0.733333,...,37.6,34.7,NaN,0.830662,0.830662,0.000000,0.640312,0.916515,0.458258,NaN
12,RandomForest,5,KuHar,0.765041,0.005077,0.764478,5.166508e-03,0.838398,0.789223,0.746354,...,38.4,35.2,NaN,0.748331,0.748331,0.806226,1.445683,1.113553,0.400000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,RandomForest,25,UCI,0.806259,0.005032,0.808513,4.665965e-03,0.777643,0.719082,0.920757,...,117.9,NaN,NaN,4.526588,4.526588,0.447214,0.943398,0.943398,NaN,NaN
243,RandomForest,50,UCI,0.794039,0.004979,0.796886,4.777976e-03,0.774622,0.700727,0.912268,...,120.8,NaN,NaN,3.130495,3.130495,0.781025,1.135782,0.871780,NaN,NaN
246,RandomForest,100,UCI,0.803130,0.003158,0.805601,3.182458e-03,0.781522,0.719324,0.922152,...,118.2,NaN,NaN,1.743560,1.743560,0.663325,1.400000,1.077033,NaN,NaN
249,RandomForest,150,UCI,0.793443,0.005504,0.797224,5.155517e-03,0.767118,0.675795,0.912941,...,112.2,NaN,NaN,4.583667,4.583667,0.979796,1.562050,1.720465,NaN,NaN


In [17]:
# Save results
import json

with open('df_results_umap_dimension.json', 'w') as file:
    json.dump(df_results.to_dict(), file)
    
with open('results_dict_umap_dimension.json', 'w') as file:
    json.dump(results_dict, file)